In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

from tensorflow.python.keras.callbacks import (
    ReduceLROnPlateau,
    EarlyStopping
)
import yaml
import random
import os

In [2]:
def seed_everything(seed=1234):
    """Set seeds for reproducibility"""
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    tf.keras.utils.set_random_seed(seed)
    tf.config.experimental.enable_op_determinism()

In [3]:
conf_fn = "model_config.yml"

In [4]:
with open(conf_fn) as cf:
    conf = yaml.load(cf, Loader=yaml.FullLoader)

In [5]:
filter1 = conf["filter1"]
filter2 = conf["filter2"]
learning_rate = conf["learning_rate"]
batch_size = conf["batch_size"]
dropout = conf["dropout"]
epochs = conf["epochs"]
seed = conf["seed"]

In [6]:
seed_everything(seed)

In [7]:
num_classes = 10
input_shape = (32, 32, 3)

# Load the data and split it between train and test sets
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()#tf.keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255

# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


# convert class vectors to binary class matrices
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

x_train shape: (50000, 32, 32, 3, 1)
50000 train samples
10000 test samples


In [8]:
model = tf.keras.Sequential(
    [
        tf.keras.Input(shape=input_shape),
        layers.Conv2D(filter1, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(filter2, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(dropout),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 30, 30, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 15, 15, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 13, 13, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 6, 6, 64)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 2304)              0         
                                                                 
 dropout (Dropout)           (None, 2304)              0

In [9]:
callbacks = [
    EarlyStopping(**conf["callbacks"]["EarlyStopping"]),
    ReduceLROnPlateau(**conf["callbacks"]["ReduceLROnPlateau"])
]

In [10]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

history = model.fit(x_train, y_train, 
                    batch_size=batch_size, 
                    epochs=epochs, 
                    callbacks=callbacks,
                    validation_split=0.2)

Epoch 1/200
1250/1250 [==============================] - 6s 3ms/step - loss: 1.5265 - accuracy: 0.4545 - val_loss: 1.3501 - val_accuracy: 0.5246 - lr: 0.0010
Epoch 2/200
1250/1250 [==============================] - 3s 2ms/step - loss: 1.2092 - accuracy: 0.5759 - val_loss: 1.1187 - val_accuracy: 0.6128 - lr: 0.0010
Epoch 3/200
1250/1250 [==============================] - 3s 2ms/step - loss: 1.0878 - accuracy: 0.6227 - val_loss: 1.0595 - val_accuracy: 0.6344 - lr: 0.0010
Epoch 4/200
1250/1250 [==============================] - 3s 2ms/step - loss: 1.0184 - accuracy: 0.6479 - val_loss: 1.0125 - val_accuracy: 0.6485 - lr: 0.0010
Epoch 5/200
1250/1250 [==============================] - 3s 2ms/step - loss: 0.9650 - accuracy: 0.6662 - val_loss: 0.9615 - val_accuracy: 0.6696 - lr: 0.0010
Epoch 6/200
1250/1250 [==============================] - 3s 2ms/step - loss: 0.9247 - accuracy: 0.6807 - val_loss: 0.9679 - val_accuracy: 0.6669 - lr: 0.0010
Epoch 7/200
1250/1250 [=============================